In [34]:
# @author: Niu
# convert the teamtrees_network.json to csv file.
# One row for each video

import json
import datetime
import dateutil
import pytz
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import csv
from datetime import datetime, timezone
from datetime import timedelta
import dateutil.parser

def get_status(sub):
    if sub>=1000000:
        return "celebrity"
    elif 10000<=sub<1000000:
        return "professional"
    else:
        return "amateur"
    
youtuber_list_path="KEY/teamtrees_network.json"

def get_video_days(day_str):
    video_date = dateutil.parser.parse(day_str)
    crawl_date=datetime(2020, 1, 8, tzinfo=timezone.utc)
    delta = crawl_date - video_date
    return delta.days

data=[]
with open(youtuber_list_path,'r') as fp:
    youtubers=json.loads(fp.read())["nodes"]
    for you in youtubers:
        channel_name=you["channel_name"]
        for video in you["video_list"]:
            del video["caption"]
            del video["comment_list"]
            video["emb_url"]="https://www.youtube.com/embed/"+video["video_id"]+"?autoplay=0"
            video["channel_name"]=channel_name
            video["tags"]=",".join(["\""+t+"\"" for t in video["tags"]])
            video["subscriber_count"]=you["subscriber_count"]
            date1=dateutil.parser.isoparse(video["publish_at"])
            date2=dateutil.parser.isoparse("2020-01-08T00:00:00.000Z")
            video["days_on"]=(date2-date1).days
            video["cel_lvl"]=get_status(you["subscriber_count"])
            video["vw_ct"]=video["view_count"]           
            video["cmnt_ct"]=video["comment_count"]
            video["day_vw"]=video["view_count"]  /get_video_days(video["publish_at"])         
            video["day_cmnt"]=video["comment_count"] /get_video_days(video["publish_at"])
            video["rel_vw"]=video["view_count"]*you["video_count"]/you["view_count"]
            video["like_rt"]=(video["like_count"]-video["dislike_count"])*100/(video["view_count"]+1)
            video["cmnt_rt"]=video["comment_count"]*100/(video["view_count"]+1)
            data.append(video)
            
        
youtuber_save_path="KEY/video_meta_data.csv"
with open(youtuber_save_path, mode='w+', encoding="utf-8", newline='') as fp:
    csv_writer = csv.writer(fp, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    first = True
    for v in data:
        if first:
            csv_writer.writerow([key for key, value in v.items()])
            first = False
        csv_writer.writerow([value for key, value in v.items()])
        
print(f"total videos:{len(data)}")
print(f"total youtubers:{len(youtubers)}")
print(f"colums:{','.join(data[0].keys())}")

total videos:518
total youtubers:473
colums:video_id,title,video_length,publish_at,category_id,category,language,live_content,description,tags,comment_count,view_count,like_count,dislike_count,emb_url,channel_name,subscriber_count,days_on,cel_lvl,vw_ct,cmnt_ct,day_vw,day_cmnt,rel_vw,like_rt,cmnt_rt


In [35]:
#@author Mai
#@version: 2/27/20

import csv
import os
import statistics
import math
from collections import defaultdict
from collections import Counter

result_path = "./KEY/"
directory="./data_tagging_j8"

data = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        with open(directory+"/"+filename, mode = "r+", encoding="utf-8") as csv_file:
            csv_reader = csv.DictReader(csv_file)
            for row in csv_reader:
                data.append(row)
    else:
        pass

wrong_data=[]
with open(result_path + "broken_data.csv", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        wrong_data.append(row)

wrong_data=[d["video_url"] for d in wrong_data]
ok = [d for d in data
      if d["STATUS"] == "OK" and d["VIDEO_URL"] not in wrong_data]

url_dict = defaultdict(list)
for d in ok:
    url_dict[d["VIDEO_URL"]].append(d)
    
manual_tag =[]
with open(result_path + "final_tag.csv", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        manual_tag.append(row)
manual_tag = [(d["VIDEO_URL"], d["final_tagging"]) for d in manual_tag]   
print(f"total accepted: {len(url_dict)}")

def convert_rating(mylist, tag):
    rating = [d[tag] for d in mylist]
    total = []
    for i in range(len(rating)):
        if rating[i] == "very_high": total.append(5)
        elif rating[i] == "high":    total.append(4)
        elif rating[i] == "neutral": total.append(3)
        elif rating[i] == "low":     total.append(2)
        elif rating[i] == "very_low": total.append(1)    
    avg = statistics.mean(total)
    return avg

creation = ["theme_creation_object", "theme_creation_performance", "knowledge"]
participation = ["planting_trees", "donating"]
connection_comment = ["spread_word", "comment_news", "live_stream_stat", "comment_video", "live_stream_game"]

def merge_cat(cat):
    
    category = "Other" #not_recognized
    
    if (cat in creation):
        category = "Creation"
    elif (cat in participation):
        category = "Participation"
    elif (cat in connection_comment):
        category = "Connection"
    return category

def isMentioned(mylist, tag):
    count_cat = sum([float(d[tag]) for d in mylist])
    if (count_cat >= MAJORITY):
        return 1
    else: return 0

url_temp = {}
rejected_url = []
URL_COUNT = 3 #number of acceptable tasks per link
MAJORITY = math.ceil(URL_COUNT/2)
celeb_name = ["mrbeast", "arborday", "teamtrees_org","noteble_youtuber","noteble_entrepreneur"]

for url, l in url_dict.items():
    category = [c["category"] for c in l]
    is_unique = len(set(category)) == len(category) #converts list to set to check if they're unique
    if (is_unique):
        for tup in manual_tag:
            if (tup[0] == url):
                cat = tup[1]
    else:
        cat = (Counter(category).most_common(1)[0])[0]
    
    url_temp[url] = {
        "VIDEO_URL": l[0]["VIDEO_URL"],
        "style": cat,
        "part_act" : merge_cat(cat),
        "quality": convert_rating(l, "quality"),
        "creativity": convert_rating(l, "creativity"),
        "awareness": convert_rating(l, "awareness")
    }
    mention_count = 0
    for name in celeb_name:
        is_mentioned = isMentioned(l,name)
        url_temp[url][name] = "y" if is_mentioned == 1 else "n" 
        mention_count+= is_mentioned
    url_temp[url]["mention_count"] = mention_count    

print(f"total processed: {len(url_dict)}")
print(f"total accepted: {len(url_temp)}")

try:
   os.mkdir(result_path)
except OSError as e:
   print("It works I guess")

with open(result_path + "final_accepted_taggings.csv", mode='w', encoding="utf-8") as csv_result:
    csv_writer = csv.writer(csv_result,
                            delimiter = ",",
                            quotechar = '"',
                            quoting = csv.QUOTE_MINIMAL,
                           lineterminator = "\n")
    isFirst=True
    for v in url_temp.values():
        if isFirst:
            csv_writer.writerow(v.keys())
            isFirst=False
        csv_writer.writerow(v.values())

total accepted: 470
total processed: 470
total accepted: 470
It works I guess


In [36]:
#@version: 2/18/20

import csv 
import os
import json
from collections import defaultdict

result_path = "./KEY/"

data = []
wrong_data=[]
with open(result_path + "video_meta_data.csv","r+", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        data.append(row)
        
print(f"total videos: {len(data)}")

with open(result_path + "broken_data.csv", encoding="utf-8") as fp:
    csv_reader = csv.DictReader(fp)
    for row in csv_reader:
        wrong_data.append(row)

wrong_data=[d["video_url"] for d in wrong_data]
data = [d for d in data if d["emb_url"] not in wrong_data]
print(f"total wrong data: {len(wrong_data)}")

url_dict = defaultdict(list)
for d in data:
    url_dict[d["emb_url"]].append(d)

# print(url_dict)

accepted_data = []
with open(result_path + "final_accepted_taggings.csv","r+", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        accepted_data.append(row)
#print(accepted_data[0].items())
#print(accepted_data)
#print(list(accepted_data[0].items())[0:3])

accepted_url = [d["VIDEO_URL"] for d in accepted_data]
print(f"total accepted: {len(accepted_url)}")


for url in url_dict:
    list_temp = []
    for d in accepted_data:
        #print(d.items())
        if (d["VIDEO_URL"] == url):
            list_temp = [tup for tup in d.items() if tup[0] != "VIDEO_URL"]                
            break
    for i in range(len(list_temp)):
            url_dict[url][0][list_temp[i][0]]= list_temp[i][1] 
#print(json.dumps(data, indent = 2))
print(f"total final data in meta: {len(url_dict)}")
#print(url_dict)

with open(result_path + "final_meta_all.csv", mode='w', encoding="utf-8") as csv_result:
    csv_writer = csv.writer(csv_result,
                            delimiter = ",",
                            quotechar = '"',
                            quoting = csv.QUOTE_MINIMAL,
                            lineterminator = "\n")

    isFirst = True
    for i in url_dict.keys():
        if isFirst:
            header = list(url_dict[i][0])
            csv_writer.writerow(header)
            isFirst = False
        csv_writer.writerow(url_dict[i][0].values())

total videos: 518
total wrong data: 48
total accepted: 470
total final data in meta: 470


In [37]:
import pandas as pd
result_path = "./KEY/"

df=pd.read_csv(result_path+"final_meta_all.csv")
df=df.loc[df.part_act!="Other"]
statues=list(df["cel_lvl"].unique())
styles=list(df["style"].unique())
for index, row in df.iterrows():
    for st in statues:
        if st==row["cel_lvl"]:
            df.loc[index, st]="y"
        else:
            df.loc[index,st]="n"
    for sl in styles:
        if sl==row["style"]:
            df.loc[index, sl]="y"
        else:
            df.loc[index,sl]="n"
df.to_csv(result_path+"final_meta_all_processed - no other + dummy.csv", index=False)